In [1]:
from trubrics.example import get_titanic_data_and_model
train_df, test_df, model = get_titanic_data_and_model()
import rich

In [2]:
from trubrics.context import DataContext

In [4]:
train_df.head()

,Sex,Embarked,Title,Pclass,Age,SibSp,Parch,Fare,Survived
58,female,S,Miss,2,5.0,1,2,27.75,1
339,male,S,Mr,1,45.0,0,0,35.50,0
138,male,S,Mr,3,16.0,0,0,9.22,0
317,male,S,Dr,2,54.0,0,0,14.00,0
827,male,C,Master,2,1.0,0,2,37.00,1


In [5]:
test_df.head()

,Sex,Embarked,Title,Pclass,Age,SibSp,Parch,Fare,Survived
751,male,S,Master,3,6.0,0,1,12.48,1
90,male,S,Mr,3,29.0,0,0,8.05,0
852,female,C,Miss,3,9.0,1,1,15.25,0
718,male,Q,Mr,3,NaN,0,0,15.50,0
36,male,C,Mr,3,NaN,0,0,7.23,1


In [3]:
data_context = DataContext(
    name="my_first_dataset",
    version="0.0.1",
    target="Survived",
    testing_data=test_df,
    training_data=train_df,
)

In [6]:
model.predict(test_df)

array([1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 0], dtype=int64)

In [7]:
from trubrics.validations import ModelValidator
from examples.classification_titanic.custom_scorer import custom_scorers  # see ./custom_scorer.py example
from examples.classification_titanic.slicing_functions import slicing_functions  # see ./slicing_functions.py for examples

In [8]:
model_validator = ModelValidator(
    data=data_context, model=model, custom_scorers=custom_scorers, slicing_functions=slicing_functions
)

In [9]:
performance = [
    # validate overall model performance with a manual threshold
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.7),

    # validate model performance on a specific data slice
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.8, data_slice="children"),

    # validate model performance with a custom metric "my_custom_loss"
    model_validator.validate_performance_against_threshold(metric="my_custom_loss", threshold=-0.7),

    # validate model performance difference between train and test sets (overfit)
    model_validator.validate_performance_between_train_and_test(metric="recall", threshold=0.3),

    # validate model performance against an sklearn dummy model
    model_validator.validate_test_performance_against_dummy(metric="accuracy"),

    # validate model performance is regular between various data slices
    model_validator.validate_performance_std_across_slices(metric="accuracy", std_threshold=0.07, dataset="testing_data", data_slices=["male", "children"]),
]

for performance_val in performance:
    rich.print(performance_val)

Validation(
    validation_type='validate_performance_against_threshold',
    validation_kwargs={'args': (), 'kwargs': {'metric': 'recall', 'threshold': 0.7}},
    passed=True,
    severity='error',
    result={'performance': 0.7184466019417476, 'sample_size': 295},
    explanation='**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on 
any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom 
trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nA
rgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed in 
when initialising the ModelValidator object.\nthreshold: the performance threshold that the model must 
attain.\ndataset: the name of a dataset from the DataContext {\'testing_data\', \'training_data\'}.\ndata_slice: 
the name of the data slice, specified in the slicing_functions parameter of ModelValidator.\nseverity: severity of 
the validation. Can be either {\'error\', \'warning\', \'experiment\'}. If None, defaults to 
\'error\'.\n\nReturns:\nTrue for success, false otherwise. With a results dictionary giving the actual model 
performance calculated.\n'
)

Validation(
    validation_type='validate_performance_against_threshold',
    validation_kwargs={'args': (), 'kwargs': {'metric': 'recall', 'threshold': 0.8, 'data_slice': 'children'}},
    passed=True,
    severity='error',
    result={'performance': 1.0, 'sample_size': 11},
    explanation='**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on 
any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom 
trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nA
rgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed in 
when initialising the ModelValidator object.\nthreshold: the performance threshold that the model must 
attain.\ndataset: the name of a dataset from the DataContext {\'testing_data\', \'training_data\'}.\ndata_slice: 
the name of the data slice, specified in the slicing_functions parameter of ModelValidator.\nseverity: severity of 
the validation. Can be either {\'error\', \'warning\', \'experiment\'}. If None, defaults to 
\'error\'.\n\nReturns:\nTrue for success, false otherwise. With a results dictionary giving the actual model 
performance calculated.\n'
)

Validation(
    validation_type='validate_performance_against_threshold',
    validation_kwargs={'args': (), 'kwargs': {'metric': 'my_custom_loss', 'threshold': -0.7}},
    passed=True,
    severity='error',
    result={'performance': -0.203, 'sample_size': 295},
    explanation='**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on 
any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom 
trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nA
rgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed in 
when initialising the ModelValidator object.\nthreshold: the performance threshold that the model must 
attain.\ndataset: the name of a dataset from the DataContext {\'testing_data\', \'training_data\'}.\ndata_slice: 
the name of the data slice, specified in the slicing_functions parameter of ModelValidator.\nseverity: severity of 
the validation. Can be either {\'error\', \'warning\', \'experiment\'}. If None, defaults to 
\'error\'.\n\nReturns:\nTrue for success, false otherwise. With a results dictionary giving the actual model 
performance calculated.\n'
)

Validation(
    validation_type='validate_performance_between_train_and_test',
    validation_kwargs={'args': (), 'kwargs': {'metric': 'recall', 'threshold': 0.3}},
    passed=True,
    severity='error',
    result={
        'train_performance': 0.9832635983263598,
        'test_performance': 0.7184466019417476,
        'train_sample_size': 596,
        'test_sample_size': 295
    },
    explanation='**Performance validation comparing training and test data scores.**\n\nScores the test set and the
train set in the DataContext, and validates whether the test score is inferior to but also within a certain range 
of the train score. Can be used to validate for overfitting\non the training set.\n\nExample:\n```py\nfrom 
trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_performance_between_train_and_test(\nmetric="recall",\nthreshold=0.3\n)\n```
\n\nArgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed 
in when initialising the ModelValidator object.\nthreshold: a positive value representing the maximum allowable 
difference between the train and test score.\ndata_slice: the name of the data slice, specified in the 
slicing_functions parameter of ModelValidator.\nseverity: severity of the validation. Can be either [\'error\', 
\'warning\', \'experiment\']. If None, defaults to \'error\'.\n\nReturns:\nTrue for success, false otherwise. With 
a results dictionary giving the model\'s performance on test and train sets.\n'
)

Validation(
    validation_type='validate_test_performance_against_dummy',
    validation_kwargs={'args': (), 'kwargs': {'metric': 'accuracy'}},
    passed=True,
    severity='error',
    result={'dummy_performance': 0.6508474576271186, 'test_performance': 0.7966101694915254, 'sample_size': 295},
    explanation='**Performance validation of testing data versus a dummy baseline model.**\n\nTrains a 
DummyClassifier / DummyRegressor from 
[sklearn](https://scikit-learn.org/stable/modules/classes.html?highlight=dummy#module-sklearn.dummy) and compares 
performance against the model on the test set.\n\nExample:\n```py\nfrom trubrics.validations import 
ModelValidator\nmodel_validator = ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_test_performance_against_dummy(\nmetric="accuracy",\nstrategy="stratified"\n
)\n```\n\nArgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer
fed in when initialising the ModelValidator object.\nstrategy: strategy of scikit-learns dummy 
model.\ndummy_kwargs: kwargs to be passed to dummy model.\ndata_slice: the name of the data slice, specified in the
slicing_functions parameter of ModelValidator.\nseverity: severity of the validation. Can be either [\'error\', 
\'warning\', \'experiment\']. If None, defaults to \'error\'.\n\nReturns:\nTrue for success, false otherwise. With 
a results dictionary giving the model\'s actual performance on the test set and the dummy model\'s performance.\n'
)

Validation(
    validation_type='validate_performance_std_across_slices',
    validation_kwargs={
        'args': (),
        'kwargs': {
            'metric': 'accuracy',
            'std_threshold': 0.07,
            'dataset': 'testing_data',
            'data_slices': ['male', 'children']
        }
    },
    passed=True,
    severity='error',
    result={
        'performances': {'testing_data_male': 0.8078817733990148, 'testing_data_children': 0.9090909090909091},
        'sample_sizes': {'testing_data_male': 203, 'testing_data_children': 11}
    },
    explanation='**Performance validation comparing data slices.**\n\nValidates that a list of model performances 
on different data slices from a given dataset has a lower\nstandard deviation than a given threshold 
value.\n\nExample:\n```py\nfrom trubrics.validations import ModelValidator\nslicing_functions = {"female": lambda 
x: x[x["Sex"]=="female"], "male": lambda x: x[x["Sex"]=="male"]}\nmodel_validator = 
ModelValidator(data=data_context, model=model, 
slicing_functions=slicing_functions)\nmodel_validator.validate_performance_std_across_slices(\nmetric="recall",\nda
taset="training_data",\ndata_slices=["male", 
"female"],\nstd_threshold=0.05,\ninclude_global_performance=True\n)\n```\n\nArgs:\nmetric: performance metric name 
defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed in when initialising the ModelValidator 
object.\ndata_slices: a list of of data slices, specified in the slicing_functions parameter of 
ModelValidator.\nstd_threshold: the standard deviation threshold that must be superior to the standard deviation of
all data slice performances.\ndataset: the name of a dataset from the DataContext {\'testing_data\', 
\'training_data\'}.\ninclude_global_performance: whether or not to include the dataset global performance in the 
list.\nseverity: severity of the validation. Can be either {\'error\', \'warning\', \'experiment\'}. If None, 
defaults to \'error\'.\n'
)

In [10]:
explainability = [
    # validate that a feature is in the top N important features of permutation feature importances
    model_validator.validate_feature_in_top_n_important_features(dataset="testing_data", feature="Sex", top_n_features=3),

    # validate the top N features are the same in train and test sets (overfit)
    model_validator.validate_feature_importance_between_train_and_test(top_n_features=2),
]

for explainability_val in explainability:
    rich.print(explainability_val)

2023-05-24 18:26:36.052 | DEBUG    | trubrics.validations.model.base:_compute_permutation_feature_importance:629 - Computing permutation feature importance for testing_data with 10 permutations.
2023-05-24 18:26:38.717 | DEBUG    | trubrics.validations.model.base:_compute_permutation_feature_importance:629 - Computing permutation feature importance for training_data with 10 permutations.


Validation(
    validation_type='validate_feature_in_top_n_important_features',
    validation_kwargs={'args': (), 'kwargs': {'dataset': 'testing_data', 'feature': 'Sex', 'top_n_features': 3}},
    passed=True,
    severity='error',
    result={
        'feature_importance_ranking': 0,
        'feature_importance': {
            'Sex': 0.08237288135593218,
            'Embarked': 0.0037288135593219972,
            'Title': 0.039322033898305075,
            'Pclass': 0.019999999999999973,
            'Age': 0.019999999999999973,
            'SibSp': 0.014237288135593208,
            'Parch': -0.000677966101694949,
            'Fare': 0.012542372881355911
        }
    },
    explanation='**Feature importance validation for top n features.**\n\nValidates that a given feature is in the 
top n most important features. For calculation of feature importance we are using sklearn\'s 
permutation_importance.\n\nExample:\n```py\nfrom trubrics.validations import ModelValidator\nmodel_validator = 
ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_feature_in_top_n_important_features(\ndataset="testing_data",\nfeature="feat
ure_a",\ntop_n_features=2,\n)\n```\n\nArgs:\nfeature: feature to assess.\ntop_n_features: the number of most 
important features that the named feature must be ranked in. E.g. if\ntop_n_features=2, the feature must be within 
the top two most important features.\ndataset: the name of a dataset from the DataContext to calculate feature 
importance on {\'testing_data\', \'training_data\'}.\npermutation_kwargs: kwargs to pass into the 
sklearn.inspection.permutation_importance function.\n\nReturns:\nTrue for success, false otherwise. With a results 
dictionary giving the actual feature importance ranking.\n'
)

Validation(
    validation_type='validate_feature_importance_between_train_and_test',
    validation_kwargs={'args': (), 'kwargs': {'top_n_features': 2}},
    passed=False,
    severity='error',
    result={
        'training_feature_importance': {
            'Parch': -0.000677966101694949,
            'Embarked': 0.0037288135593219972,
            'Fare': 0.012542372881355911,
            'SibSp': 0.014237288135593208,
            'Age': 0.019999999999999973,
            'Pclass': 0.019999999999999973,
            'Title': 0.039322033898305075,
            'Sex': 0.08237288135593218
        },
        'testing_feature_importance': {
            'Parch': 0.0241610738255033,
            'SibSp': 0.061241610738254994,
            'Embarked': 0.06493288590604022,
            'Title': 0.09161073825503355,
            'Pclass': 0.10687919463087243,
            'Age': 0.11644295302013422,
            'Fare': 0.14043624161073823,
            'Sex': 0.1583892617449664
        }
    },
    explanation="**Permutation feature importance validation between train and test sets.**\n\nValidates that the 
ranking of top n features is the same for both test and train sets. For calculation of feature importance we are 
using sklearn's permutation_importance.\n\nExample:\n```py\nfrom trubrics.validations import 
ModelValidator\nmodel_validator = ModelValidator(data=data_context, 
model=model)\nmodel_validator.validate_feature_importance_between_train_and_test(\ntop_n_features=1\n)\n```\n\nArgs
:\ntop_n_features: the number of most important features to consider for comparison between train and test sets. 
E.g. if top_n_features=2, the train and test sets must have the same 2 most important features, in the same 
order.\npermutation_kwargs: kwargs to pass into the sklearn.inspection.permutation_importance 
function.\n\nReturns:\nTrue for success, false otherwise. With a results dictionary giving the actual feature 
importance ranking.\n"
)

In [11]:
from examples.classification_titanic.custom_validator import CustomValidator  # see custom_validator.py for example

In [13]:
model_custom_validator = CustomValidator(data=data_context, model=model, custom_scorers=custom_scorers)

In [14]:
custom = [
    model_custom_validator.validate_master_age(age_limit_master=13, severity="warning")
]

rich.print(custom[0])

Validation(
    validation_type='validate_master_age',
    validation_kwargs={'args': (), 'kwargs': {'age_limit_master': 13}},
    passed=True,
    severity='warning',
    result={
        'errors_df': {
            'Sex': {},
            'Embarked': {},
            'Title': {},
            'Pclass': {},
            'Age': {},
            'SibSp': {},
            'Parch': {},
            'Fare': {},
            'Survived': {}
        }
    },
    explanation='Validate that passengers with the title "master" are younger than a certain 
age\n\nArgs:\nage_limit_master: cut off value for master\n\nReturns:\nTrue for success, false otherwise. With a 
results dictionary giving dict of errors.\n'
)

In [15]:
from trubrics.validations import Trubric

validations =  performance + explainability + custom

trubric = Trubric(
    name="classification_trubric",
    model_name="my_model",
    model_version="0.0.1",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    metadata={"some parameter": "xyz"},
    tags=["nb-demo"],
    validations=validations,
)

In [16]:
# save trubric to a local .json
trubric.save_local(path="./my_first_trubric.json")

2023-05-24 18:27:52.748 | INFO     | trubrics.validations.dataclass:save_local:120 - Trubric saved to ./my_first_trubric.json.


In [17]:
from trubrics.validations.run import TrubricRun

In [18]:
trubric_from_file = Trubric.parse_file("./my_first_trubric.json")

trubric_run_context = TrubricRun(
    data_context=data_context,
    model=model,
    trubric=trubric_from_file,
    tags=["nb-demo-new"],
    custom_validator=CustomValidator,
    custom_scorers=custom_scorers,
    slicing_functions=slicing_functions
)

In [19]:
new_trubric = trubric_run_context.set_new_trubric()

validate_performance_against_threshold [ERROR]......................................................PASS

validate_performance_against_threshold [ERROR]......................................................PASS

validate_performance_against_threshold [ERROR]......................................................PASS

validate_performance_between_train_and_test [ERROR].................................................PASS

validate_test_performance_against_dummy [ERROR].....................................................PASS

...

2023-05-24 18:28:19.286 | DEBUG    | trubrics.validations.model.base:_compute_permutation_feature_importance:629 - Computing permutation feature importance for testing_data with 10 permutations.


validate_feature_in_top_n_important_features [ERROR]................................................PASS

2023-05-24 18:28:20.666 | DEBUG    | trubrics.validations.model.base:_compute_permutation_feature_importance:629 - Computing permutation feature importance for training_data with 10 permutations.


validate_feature_importance_between_train_and_test [ERROR]..........................................FAIL

validate_master_age [WARNING].......................................................................PASS

In [22]:
new_trubric.save_local()  # save the new trubric

2023-05-24 18:29:16.835 | INFO     | trubrics.validations.dataclass:save_local:120 - Trubric saved to ./classification_trubric.json.


In [25]:
trubrics run no-save-ui run-context-path titanic-example-trubric trubric-output-file-path "cli_demo_trubric.json"

SyntaxError: invalid syntax (2578223027.py, line 1)

In [ ]:
import os
import getpass
os.environ["TRUBRICS_CONFIG_EMAIL"] = input("Enter your Trubrics account email:")
os.environ["TRUBRICS_CONFIG_PASSWORD"] = getpass.getpass("Enter your password:")
os.environ["TRUBRICS_PROJECT_NAME"] = input("Enter your Trubrics project name:")

In [27]:
data_context

DataContext(name='my_dataset', version='v0.0.1', testing_data=        Sex Embarked   Title  Pclass   Age  SibSp  Parch   Fare  Survived
751    male        S  Master       3   6.0      0      1  12.48         1
90     male        S      Mr       3  29.0      0      0   8.05         0
852  female        C    Miss       3   9.0      1      1  15.25         0
718    male        Q      Mr       3   NaN      0      0  15.50         0
36     male        C      Mr       3   NaN      0      0   7.23         1
..      ...      ...     ...     ...   ...    ...    ...    ...       ...
879  female        C     Mrs       1  56.0      0      1  83.16         1
682    male        S      Mr       3  20.0      0      0   9.22         0
509    male        S      Mr       3  26.0      0      0  56.50         1
224    male        S      Mr       1  38.0      1      0  90.00         1
386    male        S  Master       3   1.0      5      2  46.90         0

[295 rows x 9 columns], target='Survived', traini

In [23]:
model_validator = ModelValidator(
    data=data_context, model=model, custom_scorers=custom_scorers, slicing_functions=slicing_functions
)

In [24]:
performance = [
    # validate overall model performance with a manual threshold
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.7),

    # validate model performance on a specific data slice
    model_validator.validate_performance_against_threshold(metric="recall", threshold=0.8, data_slice="children"),

    # validate model performance with a custom metric "my_custom_loss"
    model_validator.validate_performance_against_threshold(metric="my_custom_loss", threshold=-0.7),

    # validate model performance difference between train and test sets (overfit)
    model_validator.validate_performance_between_train_and_test(metric="recall", threshold=0.3),

    # validate model performance against an sklearn dummy model
    model_validator.validate_test_performance_against_dummy(metric="accuracy"),

    # validate model performance is regular between various data slices
    model_validator.validate_performance_std_across_slices(metric="accuracy", std_threshold=0.07, dataset="testing_data", data_slices=["male", "children"]),
]

for performance_val in performance:
    rich.print(performance_val)

ValueError: Training data not specified in DataContext.

In [21]:
from trubrics.validations import ModelValidator
from examples.classification_titanic.custom_scorer import custom_scorers  # see ./custom_scorer.py example
from examples.classification_titanic.slicing_functions import slicing_functions  # see ./slicing_functions.py for examples

In [7]:
# 2. Build validations with ModelValidator
model_validator = ModelValidator(data=data_context, model=model)

In [8]:
validations = [
    model_validator.validate_performance_against_threshold(metric="accuracy", threshold=0.7),
    model_validator.validate_feature_in_top_n_important_features(feature="Age", top_n_features=3),
]

2023-05-24 18:06:30.350 | DEBUG    | trubrics.validations.model.base:_compute_permutation_feature_importance:629 - Computing permutation feature importance for testing_data with 10 permutations.


In [9]:
validations[0].dict()

{'validation_type': 'validate_performance_against_threshold',
 'validation_kwargs': {'args': (),
  'kwargs': {'metric': 'accuracy', 'threshold': 0.7}},
 'passed': True,
 'severity': 'error',
 'result': {'performance': 0.7966101694915254, 'sample_size': 295},
 'explanation': '**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nArgs:\nmetric: performance metric name defined in sklearn (sklearn.metrics.SCORERS) or in a custom scorer fed in when initialising the ModelValidator object.\nthreshold: the performance threshold that the model must attain.\ndataset: the name of a dataset from the DataContext {\'testing_data\', \'training_data\'}.\ndat

In [10]:
# 3. Group validations into a Trubric
trubric = Trubric(
    name="my_first_trubric",
    data_context_name=data_context.name,
    data_context_version=data_context.version,
    validations=validations,
)

In [11]:
trubric

Trubric(name='my_first_trubric', passed=None, total_passed=None, total_failed=None, failing_severity='error', data_context_name='my_dataset', data_context_version='v0.0.1', model_name=None, model_version=None, tags=[], run_by=None, git_commit=None, timestamp=None, metadata=None, validations=[Validation(validation_type='validate_performance_against_threshold', validation_kwargs={'args': (), 'kwargs': {'metric': 'accuracy', 'threshold': 0.7}}, passed=True, severity='error', result={'performance': 0.7966101694915254, 'sample_size': 295}, explanation='**Performance validation versus a fixed threshold value.**\n\nCompares performance of a model on any of the datasets in the DataContext to a hard coded threshold value.\n\nExample:\n```py\nfrom trubrics.validations import ModelValidator\nmodel_validator = ModelValidator(data=data_context, model=model)\nmodel_validator.validate_performance_against_threshold(\nmetric="recall",\nthreshold=0.8\n)\n```\n\nArgs:\nmetric: performance metric name def

In [15]:
trubric.save_local(path="./my_first_trubric.json")  # save trubric as a local .json file
# trubric.save_ui()  # or to the Trubrics platform

2023-05-24 18:08:33.160 | INFO     | trubrics.validations.dataclass:save_local:120 - Trubric saved to ./my_first_trubric.json.
